In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

np.random.seed(1234)

In [26]:
def optimal_pol(α, β, δ,
                a_min=.1, a_max=10, b_max=5, N_grid_a=50, N_grid_b=50,
                T=65-21+1, γ=3, p=.5, ρ=.03, σ=.1, Rs=1.022, Rb=1.197):
    '''
    Parameters
    ----------
    α : float 
        Demand for liquid asset
    β : float between 0 and 1
        Hyperbolic discount factor β
    δ : float between 0 and 1
        Hyperbolic discount factor δ
    a_min : float greater than 0, optional
        Lower bound of savings grid. Default is 0.1
    a_max : float greater than a_min, optional
        Upper bound of savings grid. Default is 10
    b_max : float between 0 and a_max, optional
        Upper bound of debt grid. Default is 5
    N_grid_a : int, optional
        Grid size of savings. Default is 50
    N_grid_b : int, optional
        Grid size of debts. Default is 50    
    T : int, optional
        Number of years in the lifetime consumption-saving-debt problem. Default is 23
    γ : float greater than 1, optional
        Risk aversion coefficient
    p : float between 0 and 1, optional
        Probability of income being high. Default is 0.5
    ρ : float, optional
        Income growth rate. Default is 0.03   
    σ : float, optional
        Income standard deviation
    Rs : float, optional
        Saving annual interesting rate. Default is 1.072
    Rb : float, optional
        Debt annual interesting rate. Default is 1.197
    
        
    Returns
    -------
    pol_a : ndarray
        Optimal savings a' in the next period given current state savings a
    pol_b : ndarray
        Optimal debt b' in the next period given current state savings a
    pol_c : ndarray
        Optimal consumption c in the current period given current state savings a
    '''
    
    def u(x) :
        '''
        Helper function: risk-averse utility
        '''
        return (x**(1-γ)-10)/(1-γ)
    
    def _nanargmax(arr, axis=None):
        '''
        Helpfer function: find the optimal policy
        '''
        try:
            return np.nanargmax(arr, axis)
        except ValueError:
            return np.nan

    #====================================================================
    # Simulate income paths and states
    #====================================================================
    y_eps = (np.random.randint(2,size=T)*2-1)*σ/np.sqrt(2*np.pi)

    y_pro = np.ones(T)
    for t in range(1,T):
        y_pro[t] = (1+ρ)*y_pro[t-1] + y_eps[t-1]
    
    # generate high/low states of income paths
    y_high = y_pro
    y_high[1:] = y_pro[:-1]+σ/np.sqrt(2*np.pi)

    y_low = y_pro
    y_low[1:]  = y_pro[:-1]-σ/np.sqrt(2*np.pi)
    
    #====================================================================
    # Intialize policy grids
    #====================================================================
    N_grid_tot = N_grid_a*N_grid_b

    a_grid = np.linspace(a_min, a_max, N_grid_a)
    b_grid = np.linspace(0, b_max, N_grid_b)

    # consumption and debt matrices for current period
    a0 = np.repeat(np.repeat(a_grid,N_grid_b),N_grid_tot).reshape(N_grid_tot,N_grid_tot)
    b0 = np.repeat(np.tile(b_grid,N_grid_a),N_grid_tot).reshape(N_grid_tot,N_grid_tot)

    # consumption and debt matrices for next period
    a1 = np.tile(np.repeat(a_grid,N_grid_b),N_grid_tot).reshape(N_grid_tot,N_grid_tot)
    b1 = np.tile(np.tile(b_grid,N_grid_a),N_grid_tot).reshape(N_grid_tot,N_grid_tot)

    #====================================================================
    # Initialize consumption and policy matrices
    #====================================================================
    ct_mat          = np.zeros((T, N_grid_tot, N_grid_tot))
    ct_mat_high     = np.zeros((T, N_grid_tot, N_grid_tot))
    ct_mat_low      = np.zeros((T, N_grid_tot, N_grid_tot))

    pol_func_a   = np.zeros((N_grid_tot, T))*np.nan
    pol_func_b   = np.zeros((N_grid_tot, T))*np.nan
    pol_func_c   = np.zeros((N_grid_tot, T))*np.nan

    #====================================================================
    # Backout consumption according to intertemporal budget constraint 
    #====================================================================
    # last period where b'==0: c = a + y - (a' + Rb b)/Rs
    ct_mat[-1,:,:]      = a0 + y_pro[-1]  - (a1 + Rb * b0)/Rs
    ct_mat_high[-1,:,:] = a0 + y_high[-1] - (a1 + Rb * b0)/Rs
    ct_mat_low[-1,:,:]  = a0 + y_low[-1]  - (a1 + Rb * b0)/Rs

    # first period where b==0: c = a + y -(a'-b')/Rs
    ct_mat[0,:,:]      = a0 + y_pro[0]  - (a1 - b1)/Rs
    ct_mat_high[0,:,:] = a0 + y_high[0] - (a1 - b1)/Rs
    ct_mat_low[0,:,:]  = a0 + y_low[0]  - (a1 - b1)/Rs

    # middle: c = a + y -(a' -b' + Rb b)/Rs
    for t in range(1,T-1):
        ct_mat[t,:,:]      = a0 + y_pro[t]  - (a1 - b1 + Rb * b0)/Rs
        ct_mat_high[t,:,:] = a0 + y_high[t] - (a1 - b1 + Rb * b0)/Rs
        ct_mat_low[t,:,:]  = a0 + y_low[t]  - (a1 - b1 + Rb * b0)/Rs

    # trim consumptions < 0
    ct_mat[ct_mat<0] = np.nan
    ct_mat_high[ct_mat_high<0] = np.nan
    ct_mat_low[ct_mat_low<0] = np.nan

    #====================================================================
    # Initialize the v value at retirement
    #====================================================================
    v_vals_old =20*u(a1/20)

    #====================================================================
    # Period T: just before retirement
    #====================================================================
    v_val  = u(ct_mat[-1,:,:]) + u(α * a0) + β*δ*v_vals_old
    v_val_high  = u(ct_mat_high[-1,:,:]) + u(α * a0) + δ*v_vals_old
    v_val_low   = u(ct_mat_low[-1,:,:]) + u(α * a0) + δ*v_vals_old

    vmax   = np.nanmax(v_val,1)
    vmax1h = np.nanmax(v_val_high,1)
    vmax1l = np.nanmax(v_val_low,1)
    
    # record the index of optimal (a', b')
    argvmax = np.zeros(N_grid_tot) * np.nan
    for idx in range(N_grid_tot): 
        argvmax[idx] = _nanargmax(v_val[idx,:])

        # record the optimal policies
        if ~np.isnan(argvmax[idx]):
            pol_func_a[idx,-1] = a1[idx, int(argvmax[idx])]
            pol_func_b[idx,-1] = b1[idx, int(argvmax[idx])]
            pol_func_c[idx,-1] = ct_mat[-1, idx, int(argvmax[idx])]

    #====================================================================
    # Period 0-(T-1): the rest of life
    #====================================================================
    for t in range(T-2, -1, -1):
        pol_func_a_old = pol_func_a.copy() 
        pol_func_b_old = pol_func_b.copy() 
        pol_func_c_old = pol_func_c.copy()

        v_vals_old = np.tile( p*vmax1h + (1-p)*vmax1l, (N_grid_tot,1) )

        v_val       = u(ct_mat[t,:,:]) + u(α * a0) + β*δ*v_vals_old
        v_val_high  = u(ct_mat_high[t,:,:]) + u(α * a0) + δ*v_vals_old
        v_val_low   = u(ct_mat_low[t,:,:]) + u(α * a0) + δ*v_vals_old

        vmax   = np.nanmax(v_val,1)
        vmax1h = np.nanmax(v_val_high,1)
        vmax1l = np.nanmax(v_val_low,1)

        # record the index of optimal (a', b')
        argvmax = np.zeros(N_grid_tot) * np.nan
        for idx in range(N_grid_tot): 
            argvmax[idx] = _nanargmax(v_val[idx,:])

            # record the optimal policies
            if ~np.isnan(argvmax[idx]):
                pol_func_a[idx,t] = a1[idx, int(argvmax[idx])]
                pol_func_a[idx,(t+1):] = pol_func_a_old[int(argvmax[idx]),(t+1):]

                pol_func_b[idx,t] = b1[idx, int(argvmax[idx])]
                pol_func_b[idx,(t+1):] = pol_func_b_old[int(argvmax[idx]),(t+1):]

                pol_func_c[idx,t] = ct_mat[t, idx, int(argvmax[idx])]
                pol_func_c[idx,(t+1):] = pol_func_c_old[int(argvmax[idx]),(t+1):]
    
    #====================================================================
    # Record the optimal policies given savings at the beginning
    #====================================================================
    pol_idx = [N_grid_b*i for i in range(N_grid_b)]

    pol_a = pol_func_a[pol_idx]
    pol_b = pol_func_b[pol_idx]
    pol_c = pol_func_c[pol_idx]
    
    return pol_a, pol_b, pol_c

In [53]:
pol_a, pol_b, pol_c = optimal_pol(α=.5, β=.8, δ=.95,
                                  N_grid_a=30, N_grid_b=30, a_min=.1, a_max=4, b_max=3.9,
                                 T=20)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:143: RuntimeWarning: All-NaN slice encountered
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:144: RuntimeWarning: All-NaN slice encountered
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:145: RuntimeWarning: All-NaN slice encountered
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:172: RuntimeWarning: All-NaN slice encountered
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:173: RuntimeWarning: All-NaN slice encountered
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:174: RuntimeWarning: All-NaN slice encountered


In [54]:
pol_a

array([[1.7137931 , 1.84827586, 1.98275862, 1.98275862, 2.11724138,
        2.25172414, 2.3862069 , 2.3862069 , 2.52068966, 2.65517241,
        2.52068966, 2.65517241, 2.65517241, 2.92413793, 2.78965517,
        2.92413793, 3.05862069, 3.05862069, 3.32758621, 4.        ],
       [1.7137931 , 1.84827586, 1.98275862, 1.98275862, 2.11724138,
        2.25172414, 2.3862069 , 2.3862069 , 2.52068966, 2.65517241,
        2.52068966, 2.65517241, 2.65517241, 2.92413793, 2.78965517,
        2.92413793, 3.05862069, 3.05862069, 3.32758621, 4.        ],
       [1.84827586, 1.84827586, 2.11724138, 2.11724138, 2.11724138,
        2.25172414, 2.52068966, 2.52068966, 2.52068966, 2.78965517,
        2.65517241, 2.78965517, 3.05862069, 3.05862069, 3.19310345,
        3.32758621, 3.46206897, 3.46206897, 3.46206897, 4.        ],
       [1.84827586, 1.98275862, 2.11724138, 2.11724138, 2.11724138,
        2.25172414, 2.52068966, 2.52068966, 2.52068966, 2.78965517,
        2.65517241, 2.78965517, 3.05862069, 3

In [55]:
pol_b

array([[1.21034483, 1.34482759, 1.47931034, 1.34482759, 1.34482759,
        1.47931034, 1.47931034, 1.34482759, 1.34482759, 1.47931034,
        1.21034483, 1.21034483, 1.07586207, 1.21034483, 0.94137931,
        0.80689655, 0.67241379, 0.26896552, 0.        , 0.        ],
       [1.21034483, 1.34482759, 1.47931034, 1.34482759, 1.34482759,
        1.47931034, 1.47931034, 1.34482759, 1.34482759, 1.47931034,
        1.21034483, 1.21034483, 1.07586207, 1.21034483, 0.94137931,
        0.80689655, 0.67241379, 0.26896552, 0.        , 0.        ],
       [1.21034483, 1.21034483, 1.34482759, 1.21034483, 1.07586207,
        1.21034483, 1.34482759, 1.21034483, 1.07586207, 1.21034483,
        0.94137931, 0.94137931, 1.07586207, 0.94137931, 0.94137931,
        0.80689655, 0.67241379, 0.40344828, 0.        , 0.        ],
       [1.07586207, 1.21034483, 1.34482759, 1.21034483, 1.07586207,
        1.21034483, 1.34482759, 1.21034483, 1.07586207, 1.21034483,
        0.94137931, 0.94137931, 1.07586207, 0

In [56]:
pol_c

array([[0.60738916, 0.76369238, 0.78055874, 0.69583727, 0.71396267,
        0.84041411, 0.75731461, 0.83387122, 0.85587709, 0.98632542,
        0.90734271, 1.03419536, 1.03754087, 1.06392777, 1.12406131,
        1.12409863, 1.15828262, 1.06103976, 1.09270185, 1.10679545],
       [0.74187192, 0.76369238, 0.78055874, 0.69583727, 0.71396267,
        0.84041411, 0.75731461, 0.83387122, 0.85587709, 0.98632542,
        0.90734271, 1.03419536, 1.03754087, 1.06392777, 1.12406131,
        1.12409863, 1.15828262, 1.06103976, 1.09270185, 1.10679545],
       [0.74476685, 0.76658732, 0.67489372, 0.72465501, 0.74278041,
        0.89225971, 0.80916021, 0.86268895, 0.88469483, 0.90658319,
        0.96208325, 1.0889359 , 1.09228141, 1.07261257, 1.17880185,
        1.13278343, 1.16696741, 1.20131238, 1.20705167, 1.24127821],
       [0.74766179, 0.79251012, 0.80937648, 0.72465501, 0.74278041,
        0.89225971, 0.80916021, 0.86268895, 0.88469483, 0.90658319,
        0.96208325, 1.0889359 , 1.09228141, 1